<h1> <center> Projet Scraper indeed </center> </h1>

In [2]:
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
browser = webdriver.Chrome()
browser.get('https://www.indeed.fr/')
browser.maximize_window()
from time import sleep
import re

In [3]:
def Metier(string):
    recherche = browser.find_element_by_xpath('//*[@id="text-input-what"]')
    recherche.click()
    recherche.clear()
    recherche.send_keys(string)
    sleep(3)
    recherche.send_keys(Keys.ENTER)

def Localisation(string):
    recherche = browser.find_element_by_xpath('//*[@id="where"]')
    recherche.click()
    recherche.clear()
    recherche.send_keys(string)
    sleep(3)
    recherche.send_keys(Keys.ENTER)
    
def Type_Contrat(string):
    if string.lower() == 'cdi':      
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[1]/a/span[1]").click()
    if string.lower() == 'temps plein':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[2]/a/span[1]").click()
    if string.lower() == 'stage':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[3]/a/span[1]").click()
    if string.lower() == 'cdd':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[4]/a/span[1]").click()
    if string.lower() == 'apprentissage':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[5]/a/span[1]").click()
    if string.lower() == 'contrat pro':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[6]/a/span[1]").click()
    if string.lower() == 'freelance / indépendant':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[7]/a/span[1]").click()
    if string.lower() == 'intérim':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[8]/a/span[1]").click()
    if string.lower() == 'temps partiel':
        browser.find_element_by_xpath("//*[@id='JOB_TYPE_rbo']/ul/li[9]/a/span[1]").click()    

**Liste  à remplir**

In [4]:
Metier("data scientist")
Localisation("Paris")


In [4]:
#Type_Contrat('cdi')

**Creation des listes vides pour chaque colonne**

In [26]:
import numpy as np 
Titre = []
Nom_Entreprise = []
Adresse = []
Salaire = []
Descriptif_du_poste = []
Date_de_publication = []


while True:
    annonces = browser.find_elements_by_class_name("jobsearch-SerpJobCard")
    for i in range(0,len(annonces)):
        try:
            titre = annonces[i].find_element_by_class_name('title')
            Titre.append(titre.text)
        except:
            Titre.append('vide')
        try:
            nom = annonces[i].find_element_by_class_name('company')
            Nom_Entreprise.append(nom.text)

        except:
            Nom_Entreprise.append('vide')
        try:
            adresse= annonces[i].find_element_by_class_name('location')
            Adresse.append(adresse.text)
        except:
            Adresse.append('vide')
        try:
            date = annonces[i].find_element_by_class_name('date')
            Date_de_publication.append(date.text)

        except:
            Date_de_publication.append('vide')
        try:
            salaire = annonces[i].find_element_by_class_name('salaryText')
            Salaire.append(salaire.text)
        except:
            Salaire.append('vide')
        try:
            annonces[i].find_element_by_class_name('title').click()
            sleep(2)
            descriptif= browser.find_element_by_xpath("//*[@id='vjs-content']")
            Descriptif_du_poste.append(descriptif.text)
        except:
            Descriptif_du_poste.append('vide')

    
    try:
        browser.find_element_by_xpath("//*[contains(text(), 'Suivant')]").click() 
        sleep(2)
    except:
        break

df = pd.DataFrame({"Titre":Titre,"Nom_Entreprise":Nom_Entreprise,"Adresse":Adresse,
                  "Salaire":Salaire,"Descriptif_du_poste":Descriptif_du_poste,"Date_de_publication":Date_de_publication}) 



**Scrapping et remplissage des listes utiles pour le remplissage du df par la suite**

**Remplissage du dataframe et nettoyage**

**Enregistrement des données dans BDD mongo**

**Brouillon avec beautifulsoup**

In [3]:
#test

In [21]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [69]:
URL= "https://www.indeed.fr/jobs?q=data+scientist&l=paris"
page = requests.get(URL)
soup = BeautifulSoup(page.text, "html.parser")

In [57]:
# titre du job

In [70]:
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)
titre = extract_job_title_from_result(soup)
titre

['INTERN - Data Scientist (m/w/d)',
 'Senior Backend Developer (f/m/x) (relocated to Germany)',
 'Data Scientist – Stagiaire H/F',
 'Data Scientist Assistant - H/F (stage)',
 'DATA SCIENTIST INTERN - MACHINE LEARNING - AI',
 'Data scientist Junior H/F',
 'CDI Data Analyst - Scientist - F/H',
 'STAGE - Assistant(e) Data Scientist CRM -EMEA',
 'Data scientist-(H/F)',
 'DATA SCIENTIST - H/F',
 'Stage - Assistant data scientist-(H/F)',
 'Data Scientist - Paris']

In [59]:
len(titre)

11

In [60]:
# compagny Name

In [63]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
 
compagny = extract_company_from_result(soup)

In [64]:
compagny

['Teradata',
 'Ubisoft',
 'Ubisoft',
 'Sensego',
 'Vallourec',
 'Groupe CANAL+',
 'Société Générale',
 'La Banque de France',
 'Ubisoft',
 'Fieldbox.ai',
 'Ingeniance']

In [55]:
len(compagny)

11

In [102]:
# constat : le scrapping avec beautifulsoup ne correspondent pas exactement aux elements de la page 
# pourquoi ? 
# utiliser selenium 

**Brouillon selenium**

In [ ]:
print(Titre )
print(Nom_Entreprise )
print(Adresse )
print(Salaire )
print(Descriptif_du_poste )
print(Date_de_publication )

In [ ]:
while True:
    
    titres = browser.find_elements_by_class_name('title')
    for titre in titres: 
        Titre.append(titre.text)

    noms = browser.find_elements_by_class_name('company')
    for nom in noms: 
        Nom_Entreprise.append(nom.text)
        
    adresses = browser.find_elements_by_class_name('location')
    for adresse in adresses: 
        Adresse.append(adresse.text)   
    
    dates = browser.find_elements_by_class_name('date')
    for date in dates:
        Date_de_publication.append(date.text)
        
    salaires = browser.find_elements_by_class_name('salary')
    for salaire in salaires:
        Salaire.append(salaire.text)
        
    descriptifs = browser.find_elements_by_class_name('summary')
    for descriptif in descriptifs:
        Descriptif_du_poste.append(descriptif.text)
    
    try:
        browser.find_element_by_class_name('pn').click() # 
        sleep(5)
    except:
        break